# Example Seldon Core Deployments using Helm
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

In [116]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


In [117]:
import time
from seldon_core.seldon_client import SeldonClient

def wait_for_seldon_ready(sc: SeldonClient, timeout: int = 60):
    start = time.time()
    while time.time() - start < timeout:
        r = sc.predict(transport="rest")
        if r.success:
            return
        time.sleep(2)
    raise TimeoutError("Seldon deployment not ready within timeout")

## Serve Single Model

In [118]:
!helm upgrade -i mymodel ../helm-charts/seldon-single-model --set 'model.image=seldonio/mock_classifier:1.9.0-dev' --namespace seldon

Release "mymodel" has been upgraded. Happy Helming!
NAME: mymodel
LAST DEPLOYED: Fri Nov 14 18:14:29 2025
NAMESPACE: seldon
STATUS: deployed
REVISION: 2
TEST SUITE: None


In [119]:
#!helm template mymodel ../helm-charts/seldon-single-model --set 'model.image=seldonio/mock_classifier:1.5.0-dev' | pygmentize -l json

In [97]:
!kubectl wait deployment \
  -l seldon-deployment-id=mymodel \
  --for=condition=available \
  --timeout=120s

deployment.apps/mymodel-default-0-model condition met


### Get predictions

In [120]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(
    deployment_name="mymodel",
    namespace="seldon",
    gateway_endpoint="localhost:8003",
    gateway="ambassador",
)

wait_for_seldon_ready(sc)

#### REST Request

In [121]:
r = sc.predict(transport="rest")
assert r.success == True

#### GRPC Request

In [122]:
r = sc.predict(transport="grpc")
assert r.success == True

In [123]:
!helm delete mymodel

release "mymodel" uninstalled


## Serve REST AB Test

In [124]:
!helm upgrade -i myabtest ../helm-charts/seldon-abtest --namespace seldon

Release "myabtest" has been upgraded. Happy Helming!
NAME: myabtest
LAST DEPLOYED: Fri Nov 14 18:15:04 2025
NAMESPACE: seldon
STATUS: deployed
REVISION: 4
TEST SUITE: None


In [125]:
#!helm template ../helm-charts/seldon-abtest | pygmentize -l json

In [127]:
!kubectl wait deploy \
  -l seldon-deployment-id=myabtest \
  --for=condition=available \
  --timeout=120s

deployment.apps/myabtest-default-0-classifier-1 condition met
deployment.apps/myabtest-default-1-classifier-2 condition met


### Get predictions

In [128]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(
    deployment_name="myabtest",
    namespace="seldon",
    gateway_endpoint="localhost:8003",
    gateway="ambassador",
)

wait_for_seldon_ready(sc)

#### REST Request

In [129]:
r = sc.predict(transport="rest")
assert r.success == True

#### gRPC Request

In [130]:
r = sc.predict(transport="grpc")
assert r.success == True

In [131]:
!helm delete myabtest

release "myabtest" uninstalled


## Serve REST Multi-Armed Bandit

In [132]:
!helm upgrade -i mymab ../helm-charts/seldon-mab --namespace seldon

Release "mymab" does not exist. Installing it now.
NAME: mymab
LAST DEPLOYED: Fri Nov 14 18:15:59 2025
NAMESPACE: seldon
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [133]:
#!helm template ../helm-charts/seldon-mab | pygmentize -l json

In [134]:
!kubectl wait deploy \
  -l seldon-deployment-id=mymab \
  --for=condition=available \
  --timeout=120s

deployment.apps/mymab-default-0-classifier-1 condition met
deployment.apps/mymab-default-1-classifier-2 condition met
deployment.apps/mymab-default-2-eg-router condition met


### Get predictions

In [135]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(
    deployment_name="mymab",
    namespace="seldon",
    gateway_endpoint="localhost:8003",
    gateway="ambassador",
)

wait_for_seldon_ready(sc)

#### REST Request

In [136]:
r = sc.predict(transport="rest")
assert r.success == True

#### gRPC Request

In [137]:
r = sc.predict(transport="grpc")
assert r.success == True

In [138]:
!helm delete mymab

release "mymab" uninstalled
